## Deep Dive into Convolutional Neural Nets

This tutorial is intended for people who want to deep dive into how convolutional neural nets work. We've all seen diagrams like the following in the context of convolutional neural nets:

![](img/conv_net_architecture.png)

This is great as a high level description of what is going on, but doesn't give us a detailed look at what is going on and why it works.

I'm going to assume the reader is somewhat familiar with convolutional neural net concepts; I'll cover these concepts, but quickly, referring to you to other excellent and extensive resources if you want to learn more. 

# Review of ConvNets

Lets suppose you just learned about neural networks and how fully connected layers work. Convolutional Neural Networks are a kind of neural network structure that is specialized for images. In regular neural nets, we pass in an input that has many features and let the neural net adjust its weights to learn which combinations of features are helpful for predicting the output. Convolutional neural nets are so-named because they use a special type of feature detector that is specialized for detecting features in images: a "convolutional filter". These filters can detect the presence of vertical and horizontal lines in raw images, for example; once these have been detected, they can detect more complicated combinations of features that can represent real-world shapes such as eyes, wings, or legs. 

For more, [this phenomenal essay](http://cs231n.github.io/convolutional-networks/) is the standard starting point for learning the details about convolutional neural nets.

### In the weeds

Getting into more detail: we saw that regular neural nets pass information forward by making each layer a linear combination of all the neurons in the prior layer. Convolutional neural nets instead encapsulate their weights in the filters, and then share these weights across an entire image.

For example, let's say we had a 28 x 28 image, and we were applying convolutions of size 5 x 5 to it. Let's also say we used padding, so that applying the convolution resulted in an image of the same size. Also, let's say we applied 10 filters. Then there would be 5 x 5 x 10 weights in the filters, and the output layer that results would be 28 x 28 x 10. 



# Neural network basics

That's all well and good for understanding how convolutional neural nets might make good predictions once they are trained. To understand how they learn, we'll have to:

1. Understand how neural nets propagate information between neurons that are connnected via weights, and then
2. Understand how the neurons and weights are actually connected in a convolutional neural network.

## How neural networks propagate information

To review: neural nets can be thought of as complicated mathematical functions with many parameters. The goal of neural net architectures is to find a function $f$ that maps

They are trained by the backpropagation algorithm, which iteratively:

1. Makes predictions using f_theta
2. Calculates loss
3. Calculates the derivatives of the parameters with respect to the loss
4. Updates the parameters accordingly.

For a walkthrough of how this works on a concrete example, see a talk I gave [here](https://www.youtube.com/embed/aO-RRjG45X8). 

But: how does this look from the perspective of an individual neuron? 

Let's say an individual neuron, call it $n_1$ is connected to three other neurons, call them $o_1$, $o_2$, and $o_3$. This means that each of these $o$ neurons is a linear combination of $n_1$ and some other neurons.

In addition, these $o$ neurons have gradients, we'll call them $g$ - amounts by which they ultimately affect the loss $L$. 

The neuron $n_1$ affects the output by

$$ g_1 * w_{11} + g_2 * w_{12} + g_3 * w_{13} $$

See the image below for an illustration:

![](img/neuron_gradient.png)

This is the amount, therefore, that we'll need to change the value of this neuron by: _for all of the neurons it is connected to, the sum over all of those neurons' gradients, weighted by the value of the weights connecting those neurons._

What about the weights: how much should these be updated by? Recall that in convolutional neural nets weights are shared across multiple neurons, so that we might have a situation like the following:

How should $w_{11}$ be updated in this case? Let's say that, as before, the output neurons have gradients $g_i$. These weights would be updated according to 

$$ n_1 * g_1 + n_2 * g_2 + n_3 * g_3 $$

that is, the sum over all of the "input" neurons it is connected to, weighted by the gradients of the output neurons that those input neurons were connected to. 

An illustration of this is below:

![](img/neuron_conv.png)

## How convolutional neural networks are connected

We know how to build networks with fully connected layers from scratch. But how do we do the same for convolutional networks?

There will be two components to this explanation:

1. Understanding how the neurons are connected.
2. Figuring out how to code up these connections.

Consider an image with one channel - a grayscale, 5x5 image. Suppose we want to perform a convolution on this image with 4 filters, ending up with 5 5x5 layers of neurons, each of which has been "convolved" with one of these filters. How can we do this?

$$ I = \begin{bmatrix}i_{11} & i_{12} & i_{13} & i_{14} & i_{15}\\
                      i_{21} & i_{22} & i_{23} & i_{24} & i_{25}\\
                      i_{31} & i_{32} & i_{33} & i_{34} & i_{35}\\
                      i_{41} & i_{42} & i_{43} & i_{44} & i_{45}\\
                      i_{51} & i_{52} & i_{53} & i_{54} & i_{55}\\
                      \end{bmatrix} $$

$$ F = \begin{bmatrix}f_{11}^1 & f_{12}^1 & f_{13}^1 \\
                      f_{21}^1 & f_{22}^1 & f_{23}^1 \\
                      f_{31}^1 & f_{32}^1 & f_{33}^1 \end{bmatrix} $$

$$ I = \begin{bmatrix}0 & 0 & 0 & 0 & 0 & 0 & 0 \\
                      0 & i_{11} & i_{12} & i_{13} & i_{14} & i_{15} & 0\\
                      0 & i_{21} & i_{22} & i_{23} & i_{24} & i_{25} & 0\\
                      0 & i_{31} & i_{32} & i_{33} & i_{34} & i_{35} & 0\\
                      0 & i_{41} & i_{42} & i_{43} & i_{44} & i_{45} & 0\\
                      0 & i_{51} & i_{52} & i_{53} & i_{54} & i_{55} & 0\\
                      0 & 0 & 0 & 0 & 0 & 0 & 0 \\
                      \end{bmatrix} $$

Then the top left-most element would simply be:

$ 0 * f_{11}^1 + 0 * f_{12}^1 + 0 * f_{21}^1 + i_{11} * f_{22}^1 $

The one to the right of that would be:

$ 0 * f_{11}^1 + 0 * f_{12}^1 + i_{11} * f_{21}^1 + i_{12} * f_{22}^1 $

etc.

The result will be 4 output images $O$:

$$ O_1 = \begin{bmatrix}o_{11}^1 & o_{12}^1 & o_{13}^1 & o_{14}^1 & o_{15}^1\\
                      o_{21}^1 & o_{22}^1 & o_{23}^1 & o_{24}^1 & o_{25}^1\\
                      o_{31}^1 & o_{32}^1 & o_{33}^1 & o_{34}^1 & o_{35}^1\\
                      o_{41}^1 & o_{42}^1 & o_{43}^1 & o_{44}^1 & o_{45}^1\\
                      o_{51}^1 & o_{52}^1 & o_{53}^1 & o_{54}^1 & o_{55}^1\\
                      \end{bmatrix} $$
                      
$$ O_2 = \begin{bmatrix}o_{11}^2 & o_{12}^2 & o_{13}^2 & o_{14}^2 & o_{15}^2\\
                      o_{21}^2 & o_{22}^2 & o_{23}^2 & o_{24}^2 & o_{25}^2\\
                      o_{31}^2 & o_{32}^2 & o_{33}^2 & o_{34}^2 & o_{35}^2\\
                      o_{41}^2 & o_{42}^2 & o_{43}^2 & o_{44}^2 & o_{45}^2\\
                      o_{51}^2 & o_{52}^2 & o_{53}^2 & o_{54}^2 & o_{55}^2\\
                      \end{bmatrix} $$
                      
and so on for $O_3$ and $O_4$.

### Gradients for the filters and the input pixels

The backward propogation step is where things get interesting. We need to figure out how much each pixel in the input image affects the following layer and how much each element in each filter affects the following layer.

#### Input pixels

Consider a pixel in the input image $i_{22}$. This pixel will affect the first output image in 9 different ways: one for each place it has been multiplied by in the filters:

$$o_{11}^1 = f_{33}^1 * i_{22} + ...$$
$$o_{12}^1 = f_{32}^1 * i_{22} + ...$$
$$o_{13}^1 = f_{31}^1 * i_{22} + ...$$
$$o_{21}^1 = f_{23}^1 * i_{22} + ...$$
$$o_{22}^1 = f_{22}^1 * i_{22} + ...$$
$$o_{23}^1 = f_{21}^1 * i_{22} + ...$$
$$o_{31}^1 = f_{11}^1 * i_{22} + ...$$
$$o_{32}^1 = f_{12}^1 * i_{22} + ...$$
$$o_{33}^1 = f_{13}^1 * i_{22} + ...$$

So, to code up the appropriate gradients for the input pixels, we would need to loop over the output images, and inside this loop, loop over the appropriate filter-ouptut image pairs for those input pixels (e.g. the nine filter-output image pairs above).

#### Filters

For the filters: each filter will be multiplied by almost all of the pixels in the input image and almost all of the pixels in the output image. For example, for the filter pixel $f_{11}^1$, it will be slid over all of the images in the input array, so you'll have:

$$o_{22}^1 = i_{11} * f_{11}^1 + ...$$
$$o_{23}^1 = i_{12} * f_{11}^1 + ...$$
$$o_{32}^1 = i_{21} * f_{11}^1 + ...$$
$$o_{33}^1 = i_{22} * f_{11}^1 + ...$$

To update the weights for each filter, therefore, we should simply need to loop over the appropriate input image and output image locations for that filter.

## Complications:

Now we add some complications:

### Channels

What if we have channels in the input image? The input image could be three channels, for example. 

First, the forward pass. Each neuron in the output layer would now receive a sum from multiple filters: one slid over the “red” color channel of an image at a given location, one slid over the “blue” color channel, and one slid over a “green” color channel. 

How would having three channels modify the backward pass? The gradients for each of the three image channels wouldn’t change at all - each of them would be multiplied by one filter to produce each of the output filters.

What about the gradients for the filters? These stay the same as well - except we now have three input filters connecting to four output images, so the dimensions of our filters array will be 3 x 4 x (filter height) x (filter width).

#### Illustration

There is a classic visualization from the AlexNet paper that illustrates this well. AlexNet the architecture that put convolutional neural networks on the map. It used 11x11 filters over the input image at the beginning. These 96 filters, once the network had been fully trained, are shown below:

![](img/alexnet_filters.jpeg)

If you zoom in as far as your browser will let you, you'll see that these filters are indeed 11x11 pixel color images - they each have a red, green, and blue component. 

These represent taking the 4 dimensional array containing _all_ of the trained filters, which for AlexNet would have dimensions 3 x 96 x 11 x 11, and combining each of the R, G, and B values in the 11 * 11 positions in each filter to get a "color" of that pixel.

When we consider the complexity of what is occurring here, it is remarkable that the filters that result are comprehensible!

### Batch size

What if we are feeding multiple images into the network at once, instead of one at a time?

#### Image gradients

The image gradients are still connected to the next layer in the same way; there is now just one image gradient per image, for all the images in the batch.

The main place this changes things is that now the filters have more total influence on the output. For a given batch, the filters’ changes will be based on all the images in the input batch. Thus, we'll have to sum the filters' changes across all the images in the batch and then divide by the batch size.

## Pooling

How are neurons involved in pooling connected to future layers in the network?

Consider four neurons that are combined into a single neuron via max pooling. The only neuron that actually influences the future layers in the network is the neuron that was chosen as the max. This neuron should simply be updated by whatever the gradient of the pixel it is connected to in the layer in front of it is.

## Coding this up:

There are several things to code up here. We need a 

* Convolutional forward pass
* Convolutional backward pass
* Pooling forward pass
* Pooling backward pass

### Convolutional forward pass

Should take in:

* A 4d array of images: 
    * The first dimension will be the number of images (batch size)
    * The second dimension will be the number of channels of this image
    * The third and fourth dimensions will be the height and width of this image

* A 4d array of filters:
    * The first dimension will be the number of input channels
    * The second dimension will be the number of output channels
    * The third dimension will be the filter height
    * The fourth dimension will be the filter width

It returns:

* A convout of shape:
    * The first dimension will be the number of image (batch size)
    * The second dimension will be the number of output channels
    * The third dimension will be the output height (same as input height with "same" padding)
    * The fourth dimension will be the output width (same as input width with "same" padding)

### Convolutional backward pass

Should take in:

* The same images and filters of last time
* A convout gradient with the same shape as the convout from the forward pass

And return:

* Filter gradients and image gradients of the same shape as the images and filters

### Pooling forward pass

Should take in:

* The images - the same thing conceptually as a convout

Should output:

* "poolout": the convout but with the dimensions reduced due to the stride
* An array saying which element was chosen as the maximum element (for max pooling)

### Pooling backward pass

Should take in:

* The poolout gradient
* The switches, saved from the forward pass

Should output:

* "poolout": the convout but with the dimensions reduced due to the stride
* A gradient (3/4 of which will be zero) to pass back to the images.

This implements aa convolutional forward pass:

#### Convolutional forward pass in more detail

We know that what we output should have shape:

(batch_size, channels, img_height, img_width)

A good strategy therefore will be to start with looping over the batch size and the number of channels

Then, we'll need to figure out elements in the filter and the input image to loop over to perform the convolution. When we are in the middle of the image, this will be relatively straightforward. When we are at the edge, however, we'll run into trouble. 

If our filter is 5 x 5, for example, and we are 1 pixel away from the left of the image, we'll want to exclude both the left of the filter and the left of the input image from our calculation of the output pixel's value.

For each location in the output image, therefore, we must perform an extra calculation, determining what the appropriate "x_min", "x_max", "y_min", and "y_max" to loop over are. By default, for a 5 x 5 filter, it will be -2 to 2 in both cases, but when we are near the edges we'll have to be careful. TODO: write a helper function for this

In [ ]:
def conv_bc01(imgs, filters, convout):
    """ Multi-image, multi-channel convolution
    imgs has shape (n_imgs, n_channels_in, img_h, img_w)
    For the base neural net, this is: (32, 1, 28, 28)
    filters has shape (n_channels_in, n_channels_out, img_h, img_w)
    Foar the base neural net, this is: (1, 12, 5, 5)
    convout has shape (n_imgs, n_channels_out, img_h, img_w)
    For the base neural net, this is: (32, 12, 28, 28)
    """
    # TODO: support padding and striding  
    # TODO: experiment with border mode 'reflect'  

    n_imgs = imgs.shape[0]
    img_h = imgs.shape[2]
    img_w = imgs.shape[3]
    n_channels_in = filters.shape[0]
    n_channels_out = filters.shape[1]
    fil_h = filters.shape[2]
    fil_w = filters.shape[3]
    
    fil_mid_h = fil_h // 2 # 3
    fil_mid_w = fil_w // 2 # 3

    for i in range(n_imgs): # batch size
        for c_out in range(n_channels_out): # number of filters (e.g. 12)
            for y in range(img_h): 
                y_off_min = max(-y, -fil_mid_h) # Get the minimum value of the filter
                # If y = 1, fil_mid_h = 3, so max(-1, -3) = -1, so the 
                # filter will only go one unit in the y direction.
                y_off_max = min(img_h-y, fil_mid_h+1)
                # Similarly, if we are "close to the top of the image", 
                # cut off the filter height.
                for x in range(img_w):
                    x_off_min = max(-x, -fil_mid_w)
                    # Similar logic for the left of the filter...
                    x_off_max = min(img_w-x, fil_mid_w+1)
                    # ...and the right of the filter.
                    value = 0.0
                    for y_off in range(y_off_min, y_off_max):
                        for x_off in range(x_off_min, x_off_max):
                            # Loop through the range of the filter, for:
                            # A given image.
                            # A given channel.
                            # A given height and width value in the image.
                            img_y = y + y_off
                            img_x = x + x_off
                            fil_y = fil_mid_w + y_off
                            fil_x = fil_mid_h + x_off
                            # Get the correct pixel value and the correct image value
                            for c_in in range(n_channels_in): # For each channel into the image
                                value += imgs[i, c_in, img_y, img_x] * filters[c_in, c_out, fil_y, fil_x]
                                # Add the value in:
                                # the first image, in the first pixel value, times the filter
                    # For the value in the first pixel, first channel, if the convolution size for
                    # this value is 3 x 3, then the value in cell (1,1) in the first output neuron is:
                    # imgs[1, 1, 1, 1] * filters[1, 1, 1, 1] + 
                    # imgs[1, 1, 1, 2] * filters[1, 1, 1, 2] +
                    # imgs[1, 1, 1, 3] * filters[1, 1, 1, 3] +
                    # imgs[1, 1, 2, 1] * filters[1, 1, 2, 1] +
                    # imgs[1, 1, 2, 2] * filters[1, 1, 2, 2] +
                    # imgs[1, 1, 2, 3] * filters[1, 1, 2, 3] +
                    # imgs[1, 1, 3, 1] * filters[1, 1, 3, 1] +
                    # imgs[1, 1, 3, 2] * filters[1, 1, 3, 2] +
                    # imgs[1, 1, 3, 3] * filters[1, 1, 3, 3]
                    convout[i, c_out, y, x] = value    
    
    return convout

### Convolutional backprop:

The strategy for convolutional backprop:

We need to update the filters gradient and the images gradient. To calculate these values we'll need to loop over:

1. The input images
2. The output channels
3. Just as in the forward propagation, for each (x, y) location in the image, we'll have to find the appropriate filter "min and max" positions to loop over to account for all the input-output connections.
4. Once we have found these min and max values, we loop all the filter locations, and update both the filter gradients and the image gradients appropriately, using the logic described above. 

In [2]:
def bprop_conv_bc01(imgs, convout_grad, filters, imgs_grad, filters_grad):
    """ Back-propagate gradients of multi-image, multi-channel convolution
    Inputs:
    imgs shape: (n_imgs, n_channels_in, img_h, img_w)
    filters has shape (n_channels_in, n_channels_out, img_h, img_w)
    convout_grad has same shape as convout: (n_imgs, n_channels_out, img_h, img_w)

    Returns:
    imgs_grad has same shape as imgs: (n_imgs, n_channels_in, img_h, img_w)
    filters_grad has same shape as filters: (n_channels_in, n_channels_out, img_h, img_w)

    """

    n_imgs = convout_grad.shape[0]
    img_h = convout_grad.shape[2]
    img_w = convout_grad.shape[3]
    n_channels_convout = filters.shape[1]
    n_channels_imgs = filters.shape[0]
    fil_h = filters.shape[2]
    fil_w = filters.shape[3]
    fil_mid_h = fil_h // 2
    fil_mid_w = fil_w // 2

    imgs_grad = np.zeros((n_imgs, n_channels_imgs, img_h, img_w)) # Same shape as images
    filters_grad = np.zeros((n_channels_imgs, n_channels_convout, fil_h, fil_w))  # Same shape as filters
    for i in range(n_imgs):
        for c_convout in range(n_channels_convout):
            for y in range(img_h):
                # Get the minimum and maximum indices of the convolutional filters.
                y_off_min = max(-y, -fil_mid_h)
                y_off_max = min(img_h-y, fil_mid_h+1)
                for x in range(img_w):           
                    convout_grad_value = convout_grad[i, c_convout, y, x]
                    # Get the minimum and maximum indices of the convolutional filters.
                    # indices: (image, channel_out, image "y", image "x")
                    x_off_min = max(-x, -fil_mid_w)
                    x_off_max = min(img_w-x, fil_mid_w+1)
                    for y_off in range(y_off_min, y_off_max):
                        for x_off in range(x_off_min, x_off_max):
                            img_y = y + y_off
                            img_x = x + x_off
                            fil_y = fil_mid_w + y_off
                            fil_x = fil_mid_h + x_off
                            # n_channels_imgs = channels in (e.g. 3)
                            for c_imgs in range(n_channels_imgs):
                                # for each "image" channel:
                                imgs_grad[i, c_imgs, img_y, img_x] += filters[c_imgs, c_convout, fil_y, fil_x] * convout_grad_value
                                # Add to that value in the image gradient:
                                # the sum of all the values from the filters from that particular image channel, times the gradient
                                # for this convout layer
                                filters_grad[c_imgs, c_convout, fil_y, fil_x] += imgs[i, c_imgs, img_y, img_x] * convout_grad_value
                                # As for the filters gradient: add the values in the sum of all the images:
                                # Add the sum of all the values *across all images* (as with regular gradient descent):
                                # Of: all the values that were multiplied by that weight in the forward pass, 
                                # times the convout_grad_value
    # Divide the filter_grad by the number of images
    filters_grad /= n_imgs

    
    return imgs_grad, filters_grad

Pooling: forward

In [ ]:
def pool_bc01(imgs, poolout, switches, pool_h,
              pool_w, stride_y, stride_x):
    """ Multi-image, multi-channel pooling
    imgs has shape (n_imgs, n_channels, img_h, img_w)
    poolout has shape (n_imgs, n_channels, img_h//stride_y, img_w//stride_x)
    switches has shape (n_imgs, n_channels, img_h//stride_y, img_w//stride_x, 2)
    """
    # TODO: mean pool

    n_imgs = imgs.shape[0]
    n_channels = imgs.shape[1]
    img_h = imgs.shape[2]
    img_w = imgs.shape[3]

    out_h = img_h // stride_y # 14
    out_w = img_w // stride_x # 14

    pool_h_top = pool_h // 2 - 1 + pool_h % 2 # 0 
    pool_h_bottom = pool_h // 2 + 1 # 2
    pool_w_left = pool_w // 2 - 1 + pool_w % 2 # 0
    pool_w_right = pool_w // 2 + 1 # 2

    if not n_imgs == poolout.shape[0] == switches.shape[0]:
        raise ValueError('Mismatch in number of images.')
    if not n_channels == poolout.shape[1] == switches.shape[1]:
        raise ValueError('Mismatch in number of channels.')
    if not (out_h == poolout.shape[2] == switches.shape[2] and out_w == poolout.shape[3] == switches.shape[3]):
        raise ValueError('Mismatch in image shape.')
    if not switches.shape[4] == 2:
        raise ValueError('switches should only have length 2 in the 5. dimension.')

    img_y_max = 0
    img_x_max = 0

    poolout = np.zeros((n_imgs, n_channels, out_h, out_w))
    for i in range(n_imgs):
        for c in range(n_channels):
            for y_out in range(out_h):
                y = y_out*stride_y # move along by stride_y
                # min will either 0 or (usually) the bottom of the image area
                y_min = max(y-pool_h_top, 0)
                # max will either 28 or (usually) the top of the image area
                y_max = min(y+pool_h_bottom, img_h)
                # Calculate the same for x
                for x_out in range(out_w):
                    # move along by stride_x
                    x = x_out*stride_x
                    x_min = max(x-pool_w_left, 0)
                    x_max = min(x+pool_w_right, img_w)
                    value = -9e99
                    for img_y in range(y_min, y_max):
                        for img_x in range(x_min, x_max):
                            # Within the correct image area: double for loop to find the max
                            new_value = imgs[i, c, img_y, img_x]
                            if new_value > value:
                                value = new_value
                                img_y_max = img_y
                                img_x_max = img_x
                    # Define poolout for this image and channel to be the max value
                    poolout[i, c, y_out, x_out] = value
                    # Define the switches to be the values that indices that contained the maxima
                    # in each pool.
                    switches[i, c, y_out, x_out, 0] = img_y_max
                    switches[i, c, y_out, x_out, 1] = img_x_max
                    
    return poolout, switches

Pooling: backward

In [3]:
def bprop_pool_bc01(poolout_grad, switches, imgs_grad):
    """ Multi-image, multi-channel pooling
    imgs_grad has same shape as imgs (n_imgs, n_channels, img_h, img_w)
    poolout_grad has same shape as poolout: (n_imgs, n_channels, img_h//stride_y, img_w//stride_x)
    switches has shape (n_imgs, n_channels, img_h//stride_y, img_w//stride_x, 2)
    """

    # "poolout" = "*Output* of pooling
    n_imgs = poolout_grad.shape[0] # 32
    n_channels = poolout_grad.shape[1] # 12 *number of channels of the layer "prior" to pooling*
    poolout_h = poolout_grad.shape[2] # 14
    poolout_w = poolout_grad.shape[3] # 14

#     imgs_grad = np.zeros((n_imgs, n_channels, imgs_grad.shape[2], imgs_grad.shape[3]))
    # For each image
    for i in range(n_imgs):
        # For each channel 
        for c in range(n_channels):
            for y in range(poolout_h):
                for x in range(poolout_w):
                    # Double for loop over 14 x 14
                    # Get the indices in the prior image that contained the max values.
                    img_y = switches[i, c, y, x, 0]
                    img_x = switches[i, c, y, x, 1]
                    # Make the gradient of those pixels in the images equal to
                    # the poolout gradient (the output gradient of the pooling 
                    # layer at those pixel values.
                    imgs_grad[i, c, img_y, img_x] = poolout_grad[i, c, y, x]
    return imgs_grad

## Connecting this to the rest of the neural network

We've already seen how we can define neural networks as a series of layers. For the last part, and to ensure this works, we'll want to add this in to the middle of a real neural network. 

TODO